In [1108]:
from bs4 import BeautifulSoup
import requests
import re
import csv
from datetime import datetime
import os

from PIL import Image
from io import BytesIO

import time
# from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [1124]:
class FaroesProperties:
    def __init__(self, website=None,
                 address=None, houseNum=None,
                 city=None, postNum=None,
                 price=None,LatesPrice=None, validDate=None,
                 date=None, buildingSize=None,
                 landSize=None, room=None, floor=None,
                 img=None):
        if website is None:
            self.websites = "None"
        self.websites = website
                     
        if address is None:
            self.addresses = "None"
        self.addresses = address
                     
        if houseNum is None:
            self.houseNums = "None"
        self.houseNums = houseNum

        if city is None:
            self.cities = "None"
        self.cities = city

        if postNum is None:
            self.postNums = "None"
        self.postNums = postNum
                     
        if prices is None:
            self.prices = "None"
        self.prices = price
                     
        if LatestPrices is None:
            self.LatestPrices = "None"
        self.LatestPrices = LatesPrice
                     
        if validDates is None:
            self.validDates = "None"
        self.validDates = validDate
                     
        if dates is None:
            self.dates = "None"
        self.dates = date
                     
        if buildingSizes is None:
            self.buildingSizes = "None"
        self.buildingSizes = buildingSize
                     
        if landSize is None:
            self.landSizes = "None"
        self.landSizes = landSize
                     
        if rooms is None:
            self.rooms = "None"
        self.rooms = room
                     
        if floors is None:
            self.floors = "None"
        self.floors = floor

        if img is None:
            self.imgs = "None"
        self.imgs = img
                     
    def display(self):
        print("Heimasída: ", self.websites, 
              "\nAddressa: ", self.addresses,
              "\nHúsnummar: ",self.houseNums,
              "\nPostnr: ", self.postNums,
              "\nBýur: ", self.cities,
              "\nPrísur: ", self.prices,
              "\nSeinasti bod: ",self.LatestPrices, 
              "\nGaldandi til dato: ", self.validDates,
              "\nDato: ", self.dates,
              "\nm2 í húsinum: ", self.buildingSizes,"\nm2 á økinum: ", self.landSizes,
              "\nRúm: ", self.rooms,
              "\nHæddir: ", self.floors, 
              "\nURL img ", self.imgs,
              "\n")
    def writeToCSV(self):
        print("test")
        
    def getImgUrl(self):
        if self.imgs != "None":
            return str(self.imgs)
    def getName(self):
        if self.addresses != None and self.houseNums != None and  self.cities != None:
            return str(self.cities+_+self.addresses+self.houseNums)

In [1125]:
betriURL = 'https://www.betriheim.fo/'
skynURL = 'https://www.skyn.fo/ognir-til-soelu'
skynimgs = []
betriimgs = []
def scrape_page(driver,website):
    url = f"{website}"
    driver.get(url)
    time.sleep(2)  # Give the page time to load
    return driver.page_source

# Starts up chrome
# driver = webdriver.Chrome()  

options = Options()
options.add_argument("--headless")
options.add_argument("--window-size=1920x1080")
driver = webdriver.Chrome(options=options)

try:  
    betriResponse = scrape_page(driver,betriURL)
    skynResponse = scrape_page(driver,skynURL)
finally:
    driver.quit()     

# betriResponse = requests.get(betriURL)
betriSoup = BeautifulSoup(betriResponse, 'html.parser')


# skynResponse = requests.get(skynURL)
skynSoup = BeautifulSoup(skynResponse, 'html.parser')

properties = []

In [1126]:
BetriPropertyWrappers = betriSoup.find_all('article', class_="c-property c-card grid")

SkynPropertyWrappers = skynSoup.find_all('div', class_="col-md-6 col-sm-6 col-xs-12 col-lg-4 ogn")
SkynPropertyWrappersNewbid = skynSoup.find_all('div', class_="col-md-6 col-sm-6 col-xs-12 col-lg-4 ogn newbid")
SkynPropertyWrappersNewProp = skynSoup.find_all('div', class_="col-md-6 col-sm-6 col-xs-12 col-lg-4 ogn newprop")
SkynPropertyWrappersNewPrice = skynSoup.find_all('div', class_="col-md-6 col-sm-6 col-xs-12 col-lg-4 ogn newprice")
SkynPropertyWrappersSold = skynSoup.find_all('div', class_="col-md-6 col-sm-6 col-xs-12 col-lg-4 ogn sold")
SkynPropertyWrappersFixedPrice = skynSoup.find_all('div', class_="col-md-6 col-sm-6 col-xs-12 col-lg-4 ogn fixedprice")

In [1127]:
def CheckValueText(property, attribute, classVal):
    value = property.find(attribute, class_=classVal)
    if value is not None:
        value = value.text
    else:
        value = None
    return value

In [1128]:
def addressParse(addresses):
    if str(addresses):
        pattern1 = r"((\w+ )(\w+$))"
        pattern2 = r"(\w+[ ]?[^0-9]+[ ]?[^0-9]+)"
        pattern3 = r"([0-9]+\w?)"
        match1 = re.search(pattern1, addresses)
        match2 = re.search(pattern2, addresses)
        match3 = re.search(pattern3, addresses)
        if match1:
            city = match1.group(3)
            yield city
            postNum = match1.group(2)
            yield postNum
        if match2:
            address = match2.group(1)
            yield address
        if match3:
            houseNum = match3.group(1)
            yield houseNum     

In [1129]:
def priceParse(prices):
    if prices:
        pattern = r"(Kr. )"
        match = re.sub(pattern, "", prices)
        pattern = r"(\.)"
        match = re.sub(pattern, "", match)
        return int(match)

In [1130]:
def latestPriceOfferParse(latestPriceOffer):
    if latestPriceOffer:
        pattern = r"(Seinasta boð: Kr. )"
        match = re.sub(pattern, "", latestPriceOffer)
        pattern = r"(\.)"
        match = re.sub(pattern, "", match)
        return int(match)

In [1131]:
def convert_to_date(last_two_words):
    if last_two_words:
        try:
            # Assuming the last two words are in the format "dd-mm-yyyy, kl HH:MM"
            date_obj = datetime.strptime(last_two_words, "%d-%m-%Y, kl %H:%M")
            return date_obj
        except ValueError:
            return "Invalid date format"

def priceOfferValidDateParse(priceOfferValidDate):
    if priceOfferValidDate:
        pattern = r"(Galdandi til )"
        match = re.sub(pattern, "", priceOfferValidDate)
        formatted_date = convert_to_date(match)
        return formatted_date
        
def priceOfferValidDateParseSkyn(priceOfferValidDate):
    if priceOfferValidDate:
        pattern = r"(galdandi til )"
        match = re.sub(pattern, "", priceOfferValidDate)
        # formatted_date = convert_to_date(match)
        if match:
            try:
                # Assuming the last two words are in the format "dd-mm-yyyy, kl HH:MM"
                date_obj = datetime.strptime(match, "%d.%m.%Y %H:%M")
                return date_obj
            except ValueError:
                return "Invalid date format"
            return formatted_date

In [1132]:
def convertToDigits(yearBuilt):
    if yearBuilt:
        pattern = r"([0-9]+)"
        formatted_year = re.search(pattern, yearBuilt)
        # pattern = r"()"
        # formatted_year = re.sub(pattern, "", match)
        return int(formatted_year.group(1))

In [1133]:
for property in BetriPropertyWrappers:
    websites = "Betri"
    
    addresses = CheckValueText(property, 'address', "medium")
    addressEntries = addressParse(addresses)
    city = next(addressEntries, None)
    postNum = next(addressEntries, None)
    address = next(addressEntries, None)
    houseNum = next(addressEntries, None)
    # print(houseNum, "---", postNum)
    
    
    prices = CheckValueText(property, 'div', "price")
    priceEntries = int(priceParse(prices))
    
    latestPriceOffer = CheckValueText(property, 'div', "latest-offer")
    latestPriceOfferEntries = latestPriceOfferParse(latestPriceOffer)
    
    priceOfferValidDate = CheckValueText(property, 'div', "valid")
    priceOfferValidDateEntry = priceOfferValidDateParse(priceOfferValidDate)
    
    yearBuilt = CheckValueText(property, 'div', "date")
    yearBuiltEntry = convertToDigits(yearBuilt)
    
    insideM2 = CheckValueText(property, 'div',"building-size")
    insideM2Entry = convertToDigits(insideM2)
    
    outsideM2 = CheckValueText(property, 'div', "land-size")
    outsideM2Entry = convertToDigits(outsideM2)
    
    rooms = CheckValueText(property, 'div', "rooms")
    roomEntry = convertToDigits(rooms)
    
    floors = CheckValueText(property, 'div', "floors")
    floorEntry = convertToDigits(floors) 
    
    # print(address,"-",houseNum,"-",postNum,"-",city,"-",priceEntries,"-"
    #           ,latestPriceOfferEntries,"-",priceOfferValidDateEntry,"-",yearBuiltEntry
    #           ,"-",insideM2Entry,"-",outsideM2Entry,"-",roomEntry,"-",floorEntry)
    
    # Find the first <li> element with class "slide"
    first_slide = property.find('li', class_='slide')
    if first_slide:
    #     # Find the <img> element within the first slide
        img_element = first_slide.find('img')
        if img_element:
            # Extract the 'src' attribute from the <img> element
            image_url = img_element['src']
            betriimgs.append(image_url)

    prop = FaroesProperties(websites, address, houseNum, city, postNum,
                            priceEntries,latestPriceOfferEntries, priceOfferValidDateEntry,yearBuiltEntry,insideM2Entry,
                            outsideM2Entry,roomEntry,floorEntry,image_url)
    
    
    properties.append(prop)


In [1134]:
def addressParseSkyn(addresses):
    if addresses:
        pattern1 = r"(\w+[ ]?[^0-9]+[ ]?[^0-9]+)"
        pattern2 = r"([0-9]+\w?)"
        match1 = re.search(pattern1, addresses)
        match2 = re.search(pattern2, addresses)
        if match1:
            address = match1.group(1)   
            yield str(address) 
        if match2:
            houseNum = match2.group(1)
            yield str(houseNum)
        

In [1135]:
def removedot(price):
    if price:
        pattern = r"(\.)"
        match = re.sub(pattern, "",price)
        if match:
            prices = match
            return int(prices)

In [1136]:
def removeM2(sizeinM2):
    if sizeinM2 == '−':
        return None
    if sizeinM2 == '−                \t\t':
        return None
    if sizeinM2:
        pattern = r"( m2)"
        match = re.sub(pattern, "",sizeinM2)
        if match:
            size = match
            return int(size)

In [1137]:
def skynPropertyScraper(SkynPropertyWrappers,slag):
    for property in SkynPropertyWrappers:
        websites = slag
        addresses = CheckValueText(property, 'div', "ogn_headline")    
        addressEntries = addressParseSkyn(addresses)
        address = next(addressEntries, None)
        houseNum = next(addressEntries, None)
        city = CheckValueText(property, 'div', "ogn_adress")  
        postNum = None
        
        prices = CheckValueText(property, 'div', "listprice")
        priceEntries = int(priceParse(prices))
        
        
        LatestPrices = CheckValueText(property, 'div', "latestoffer")
        latestPriceOfferEntries = removedot(LatestPrices)
        
        validDates = CheckValueText(property, 'div', "validto")
        priceOfferValidDateEntry = priceOfferValidDateParseSkyn(validDates)
        
        # Find all div elements with class "col-xs-2 text-justify"
        buildingSizes = CheckValueText(property, 'div',"col-xs-2 col-xs-offset-1 text-justify")
        insideM2Entry = removeM2(buildingSizes)
        
        div_elements = property.find_all('div', class_='col-xs-2 text-justify')
        
        outsideM2Entry = removeM2(div_elements[0].get_text(strip=True))
        rooms = div_elements[1].get_text(strip=True)
        roomEntry = None if rooms == '−' else rooms
        floors = div_elements[2].get_text(strip=True)
        floorEntry = None if floors == '−' else floors
        yearBuilt = div_elements[3].get_text(strip=True)
        yearBuiltEntry = None if yearBuilt == '−' else yearBuilt

        # Find the first <li> element with class "slide"
        first_slide = property.find('div', class_='ogn_thumb')
        if first_slide:
        #     # Find the <img> element within the first slide
            img_element = first_slide.find('img')
        if img_element:
            # Extract the 'src' attribute from the <img> element
            image_url = img_element['src']
            imgUrl = str("https://www.skyn.fo"+image_url)
            # betriimgs.append(image_url)
            skynimgs.append("https://www.skyn.fo"+image_url)
        
        
        prop = FaroesProperties(websites, address, houseNum, city, postNum,
                            priceEntries,latestPriceOfferEntries, priceOfferValidDateEntry,yearBuiltEntry,insideM2Entry,
                            outsideM2Entry,roomEntry,floorEntry,imgUrl)

        properties.append(prop)

       

skynPropertyScraper(SkynPropertyWrappers,"Skyn")
skynPropertyScraper(SkynPropertyWrappersSold,"Skyn: Selt")
skynPropertyScraper(SkynPropertyWrappersNewbid,"Skyn: Nyggj bod")
skynPropertyScraper(SkynPropertyWrappersNewProp, "Skyn: Nyggj ogn")
skynPropertyScraper(SkynPropertyWrappersNewPrice, "Skyn: Nytt bod")
skynPropertyScraper(SkynPropertyWrappersFixedPrice, "Skyn: Fasturprisur")


    # folder_path = "images/"
    # if not os.path.exists(folder_path):
    #     os.makedirs(folder_path)
    # image_name = os.path.basename(image_url)
    # print(image_name)    


In [1138]:
file = open('export_data.csv', 'w', newline='')
writer = csv.writer(file)
headers = ['website', 
           'address', 
           'houseNr',
           'postnum',
           'city', 
           'suggest_price',
           'latest_price',
           'priceOfferDueDate',
           'yearbuilt', 
           'insidem2', 
           'outsidem2', 
           'roomqty', 
           'floorlevels',
           'img_urls'
          ]
writer.writerow(headers)
for prop in properties:
    file = open('export_data.csv', 'a', newline='', encoding='utf-8')
    writer = csv.writer(file)
    headers = ([prop.websites, 
                prop.addresses, 
                prop.houseNums, 
                prop.postNums, 
                prop.cities,
                prop.prices, 
                prop.LatestPrices,
                prop.validDates,
                prop.dates, 
                prop.buildingSizes, 
                prop.landSizes, 
                prop.rooms, 
                prop.floors,
                prop.imgs
               ])
    writer.writerow(headers)
    file.close()


In [1141]:
# Determine the destination folder
folder_path = "images"

# Create the folder if it doesn't exist
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
        
for prop in properties:
    # URL of the image
    image_url = prop.getImgUrl()
    img_name = prop.getName()
    if img_name == None:
        img_name = "unknown"

    # Send an HTTP GET request to the image URL
    response = requests.get(image_url)
    
    if response.status_code == 200:
        # Read the image content
        image_data = response.content
        
        # Determine the file format (PNG or JPEG) based on the 'Content-Type' header
        content_type = response.headers.get('Content-Type', '').lower()
        if 'image/jpeg' in content_type:
            file_extension = 'jpeg'
        elif 'image/png' in content_type:
            file_extension = 'png'
        else:
            print("Unknown image format")
            file_extension = None
        
        if file_extension:
            # Create a PIL Image from the image data
            image = Image.open(BytesIO(image_data))

            # Define the full path for saving the image inside the "images" folder
            image_filename = os.path.join(folder_path, img_name+'.'+ file_extension)
            
            # Save the image to the specified folder
            image.save(image_filename)
            
            # Save the image to a file with the appropriate extension
            print("Image saved as "+img_name+"." + file_extension)
    else:
        print("Failed to download image. Status code:", response.status_code)

Image saved as TórshavnHeimaratún 3.jpeg
Image saved as VelbastaðurGerðisbreyt 3.jpeg
Image saved as TórshavnGrásteinsgøta 2B.jpeg
Image saved as FuglafjørðurKráarvegur 20.jpeg
Image saved as SørvágurLiðabrekkan 15.jpeg
Image saved as SandavágurSteigarbrekka 14.jpeg
Image saved as SkopunGarðavegur 12.jpeg
Image saved as TórshavnStoffalág 59.jpeg
Image saved as KlaksvíkGarðavegur 70.jpeg
Image saved as VágurBøgøta 30.jpeg
Image saved as HoyvíkKarlamagnusarbreyt 44.jpeg
Image saved as SaltangaráÆrgisbrekka 1.jpeg
Image saved as TórshavnNiels Winthersgøta 9.jpeg
Image saved as OyrarbakkiNeyst Matr nr 141.jpeg
Image saved as MiðvágurMatr nr 13d.jpeg
Image saved as VelbastaðurGrundstykki  176.jpeg
Image saved as SignabøurNeyst - matr. 283e.jpeg
Image saved as KlaksvíkGarðavegur 50.jpeg
Image saved as NorðragøtaÍbúð 1.jpeg
Image saved as ToftirLýðarsvegur 48B.jpeg
Image saved as NorðragøtaÍbúð 2.jpeg
Image saved as NorðragøtaÍbúð 3.jpeg
Image saved as NorðragøtaÍbúð 4.jpeg
Image saved as Tór